In [ ]:
# Clustering of timeseries data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('../../Group Project Stage/snapshot_full_df.csv')

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)x

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,store_key,sku_key,tran_date,sales,selling_price,avg_discount,store_region,store_grading,sku_department,sku_subdepartment,sku_category,sku_subcategory
0,62,35681,2017-03-22,0,100.0,0.0,KWA,B,TY,TY,000,000
1,62,35681,2017-04-22,6,100.0,0.0,KWA,B,TY,TY,000,000
2,62,35681,2017-07-06,0,100.0,0.0,KWA,B,TY,TY,000,000
3,62,35681,2017-08-15,0,100.0,0.0,KWA,B,TY,TY,000,000
4,62,35681,2017-09-02,0,100.0,0.0,KWA,B,TY,TY,000,000


In [28]:
aggregate = df.groupby(['sku_key', 'tran_date']).sum()
aggregate.drop('store_key', axis=1, inplace=True)
aggregate.reset_index(inplace=True)

In [30]:
categorical = df[['sku_key', 'sku_department',
                   'sku_subdepartment', 'sku_category', 'sku_subcategory']]

In [59]:
nw_df = pd.DataFrame([], columns=['sku_key','sku_department','sku_subdepartment','sku_category','sku_subcategory'])
for i in categorical['sku_key'].unique():
    nw_df = pd.concat([nw_df, pd.DataFrame(categorical[categorical['sku_key'] == i].iloc[0]).T])

In [60]:
nw_df.reset_index(inplace=True, drop=True)

In [70]:
aggregate['sku_key'] = aggregate['sku_key'].astype(int)
nw_df['sku_key'] = nw_df['sku_key'].astype(int)

In [72]:
aggregate_df = aggregate.merge(nw_df, how='left', on='sku_key')

In [73]:
aggregate_df.head()

,sku_key,tran_date,sales,selling_price,avg_discount,sku_department,sku_subdepartment,sku_category,sku_subcategory
0,28,2016-03-02,1,89.9,0.0,TY,06,063,20G
1,28,2016-03-03,0,89.9,0.0,TY,06,063,20G
2,28,2016-03-04,0,89.9,0.0,TY,06,063,20G
3,28,2016-03-05,0,89.9,0.0,TY,06,063,20G
4,28,2016-03-06,0,89.9,0.0,TY,06,063,20G


In [74]:
aggregate_df.to_csv('aggregate_products.csv')